# Reference Summary

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import nltk
import pandas as pd
nltk.download('punkt_tab')

# Ensure required NLTK resources are available
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Function to summarize text using word frequency
def summarize_text(text, num_sentences=3):
    # Tokenize sentences and words
    sentences = sent_tokenize(text)
    words = word_tokenize(text.lower())

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words and word not in string.punctuation]

    # Compute word frequency
    freq_dist = FreqDist(words)

    # Score sentences based on word frequency
    sentence_scores = {}
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in freq_dist:
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = freq_dist[word]
                else:
                    sentence_scores[sentence] += freq_dist[word]

    # Select top sentences for the summary
    summarized_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    return ' '.join(summarized_sentences)

data=pd.read_csv('/content/news_data_cleaned.csv')
# Generate summaries for the dataset
data['Summary'] = data['Cleaned Article Text'].apply(lambda text: summarize_text(text))

# Save the dataset with summaries to a new CSV file
output_file_path = 'reference_news_data_with_summaries.csv'
data.to_csv(output_file_path, index=False)

output_file_path



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'reference_news_data_with_summaries.csv'

# T5 model

In [ ]:
pip install transformers torch pandas


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd

# Load the pretrained T5 model and tokenizer
model_name = "t5-small"  # Use 't5-base' or 't5-large' for larger models
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to summarize text
def summarize_text_t5(text, max_input_length=512, max_output_length=150):
    input_text = f"summarize: {text}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_input_length, truncation=True)
    output_ids = model.generate(input_ids, max_length=max_output_length, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Load your dataset
file_path = "reference_news_data_with_summaries.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Apply summarization to each article
data['Summary_t5small'] = data['Cleaned Article Text'].apply(summarize_text_t5)

# Save the summarized data
output_file_path = "reference_news_data_with_summaries.csv"
data.to_csv(output_file_path, index=False)
print(f"Summarized file saved at: {output_file_path}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Summarized file saved at: reference_news_data_with_summaries.csv


In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
model_name = "t5-small"  # Use 't5-base' or 't5-large' for larger models
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
# Save the model
model_save_path = "t5-smal"
model.save_pretrained(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(model_save_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

('t5-smal/tokenizer_config.json',
 't5-smal/special_tokens_map.json',
 't5-smal/spiece.model',
 't5-smal/added_tokens.json')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd

# Load the pretrained T5 model and tokenizer
model_name = "t5-base"  # Use 't5-base' or 't5-large' for larger models
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to summarize text
def summarize_text_t5(text, max_input_length=512, max_output_length=150):
    input_text = f"summarize: {text}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_input_length, truncation=True)
    output_ids = model.generate(input_ids, max_length=max_output_length, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Load your dataset
file_path = "reference_news_data_with_summaries.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Apply summarization to each article
data['Summary_t5base'] = data['Cleaned Article Text'].apply(summarize_text_t5)

# Save the summarized data
output_file_path = "reference_news_data_with_summaries.csv"
data.to_csv(output_file_path, index=False)
print(f"Summarized file saved at: {output_file_path}")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Summarized file saved at: reference_news_data_with_summaries.csv


In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
model_name = "t5-base"  # Use 't5-base' or 't5-large' for larger models
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
# Save the model
model_save_path = "t5-base"
model.save_pretrained(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(model_save_path)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

('t5-base/tokenizer_config.json',
 't5-base/special_tokens_map.json',
 't5-base/spiece.model',
 't5-base/added_tokens.json')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd

# Load the pretrained T5 model and tokenizer
model_name = "t5-large"  # Use 't5-base' or 't5-large' for larger models
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to summarize text
def summarize_text_t5(text, max_input_length=512, max_output_length=150):
    input_text = f"summarize: {text}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_input_length, truncation=True)
    output_ids = model.generate(input_ids, max_length=max_output_length, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Load your dataset
file_path = "reference_news_data_with_summaries.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Apply summarization to each article
data['Summary_t5large'] = data['Cleaned Article Text'].apply(summarize_text_t5)

# Save the summarized data
output_file_path = "reference_news_data_with_summaries.csv"
data.to_csv(output_file_path, index=False)
print(f"Summarized file saved at: {output_file_path}")

Summarized file saved at: reference_news_data_with_summaries.csv


In [7]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
model_name = "t5-large"  # Use 't5-base' or 't5-large' for larger models
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
# Save the model
model_save_path = "t5-large"
model.save_pretrained(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(model_save_path)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

('t5-large/tokenizer_config.json',
 't5-large/special_tokens_map.json',
 't5-large/spiece.model',
 't5-large/added_tokens.json')

In [ ]:
pip install rouge-score nltk pandas


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e2b3a8604f7f48087d901dbbdb6163830f0f57463543fb01bdb599d98233712d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import pandas as pd
from rouge_score import rouge_scorer

# Load the dataset
reference_file_path = "/content/news_data_with_rouge_scores (1).csv"  # Replace with your file path
data = pd.read_csv(reference_file_path)

# Define the reference column and T5-generated summary columns
reference_column = "Summary"  # Reference summaries
generated_columns = ["Summary_t5small", "Summary_t5base", "Summary_t5large"]  # T5 model summaries

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to calculate ROUGE scores
def calculate_rouge(reference, generated):
    """
    Calculate ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    if pd.isnull(reference) or pd.isnull(generated):
        return 0.0, 0.0, 0.0  # Handle missing values gracefully

    scores = scorer.score(reference, generated)
    rouge1 = scores['rouge1'].fmeasure
    rouge2 = scores['rouge2'].fmeasure
    rougeL = scores['rougeL'].fmeasure

    return rouge1, rouge2, rougeL

# Process each T5 model column
for gen_col in generated_columns:
    if gen_col in data.columns:
        print(f"Calculating ROUGE scores for {gen_col}")
        scores = data.apply(lambda row: calculate_rouge(row[reference_column], row[gen_col]), axis=1)

        # Add new columns for each ROUGE score
        data[f'ROUGE-1_{gen_col}'] = scores.apply(lambda x: x[0])
        data[f'ROUGE-2_{gen_col}'] = scores.apply(lambda x: x[1])
        data[f'ROUGE-L_{gen_col}'] = scores.apply(lambda x: x[2])
    else:
        print(f"Column {gen_col} not found in the dataset.")

# Save the results to a new CSV file
output_file_path = "/content/news_data_with_rouge_scores (1).csv"
data.to_csv(output_file_path, index=False)

print(f"ROUGE scores saved to: {output_file_path}")



Calculating ROUGE scores for Summary_t5small
Calculating ROUGE scores for Summary_t5base
Calculating ROUGE scores for Summary_t5large
ROUGE scores saved to: /content/news_data_with_rouge_scores (1).csv


## T5-base

In [ ]:
!pip install datasets transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install wandb

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Load dataset
input_csv_path = "reference_news_data_with_summaries (2).csv"  # Your dataset path
text_column = "Cleaned Article Text"  # Column with the input text
summary_column = "Summary_t5small"  # Column with the target summaries

# Read CSV and prepare data
df = pd.read_csv("/content/t5-small-fine-tuned/reference_news_data_with_summaries (2).csv")
#df = df[[text_column, summary_column]].dropna()  # Drop rows with missing values

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Tokenizer and model initialization
model_name = "t5-small"  # Change to t5-base or t5-large for larger models
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenization function
def preprocess_function(batch):
    inputs = ["summarize: " + text for text in batch[text_column]]
    model_inputs = tokenizer(
        inputs, max_length=512, padding="max_length", truncation=True
    )

    labels = tokenizer(
        batch[summary_column], max_length=150, padding="max_length", truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split dataset into train and validation sets
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps=500,
    logging_first_step=True,
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    fp16=torch.cuda.is_available(),
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)
from transformers import TrainingArguments, Trainer

# Updated TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",  # Replaced evaluation_strategy
    eval_steps=500,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps=500,
    logging_first_step=True,
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    fp16=torch.cuda.is_available(),
)

# Disable W&B if you don't want tracking
import os
os.environ["WANDB_DISABLED"] = "true"

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,  # Deprecated in future versions; safe for now
)


# Train model
trainer.train()




Map:   0%|          | 0/90 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-17-ed83c1263947>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-17-ed83c1263947>:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Tr

Model fine-tuned and saved successfully!


In [12]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model and tokenizer
model_name = "./fine_tuned_t5"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Summarize text
def generate_summary(text):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", truncation=True, max_length=512)
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=150,
        min_length=30,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Load your dataset
file_path = "/content/t5-small-fine-tuned/reference_news_data_with_summaries (2) (1).csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Apply summarization to each article
data['Summary_t5small_finetunned'] = data['Cleaned Article Text'].apply(generate_summary)

# Save the summarized data
output_file_path = "/content/t5-small-fine-tuned/reference_news_data_with_summaries (2) (1).csv"
data.to_csv(output_file_path, index=False)
print(f"Summarized file saved at: {output_file_path}")


OSError: Incorrect path_or_model_id: './fine_tuned_t5'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
import pandas as pd
from rouge_score import rouge_scorer

# Load the dataset
reference_file_path = "/content/news_data_with_rouge_scores (2).csv"  # Replace with your file path
data = pd.read_csv(reference_file_path)

# Define the reference column and T5-generated summary columns
reference_column = "Summary"  # Reference summaries
generated_columns = ["Summary_t5small_finetunned"]  # T5 model summaries

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to calculate ROUGE scores
def calculate_rouge(reference, generated):
    """
    Calculate ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    if pd.isnull(reference) or pd.isnull(generated):
        return 0.0, 0.0, 0.0  # Handle missing values gracefully

    scores = scorer.score(reference, generated)
    rouge1 = scores['rouge1'].fmeasure
    rouge2 = scores['rouge2'].fmeasure
    rougeL = scores['rougeL'].fmeasure

    return rouge1, rouge2, rougeL

# Process each T5 model column
for gen_col in generated_columns:
    if gen_col in data.columns:
        print(f"Calculating ROUGE scores for {gen_col}")
        scores = data.apply(lambda row: calculate_rouge(row[reference_column], row[gen_col]), axis=1)

        # Add new columns for each ROUGE score
        data[f'ROUGE-1_{gen_col}'] = scores.apply(lambda x: x[0])
        data[f'ROUGE-2_{gen_col}'] = scores.apply(lambda x: x[1])
        data[f'ROUGE-L_{gen_col}'] = scores.apply(lambda x: x[2])
    else:
        print(f"Column {gen_col} not found in the dataset.")

# Save the results to a new CSV file
output_file_path = "/content/news_data_with_rouge_scores (2).csv"
data.to_csv(output_file_path, index=False)

print(f"ROUGE scores saved to: {output_file_path}")



Calculating ROUGE scores for Summary_t5small_finetunned
ROUGE scores saved to: /content/news_data_with_rouge_scores (2).csv
